# Day 5: Print Queue 
```
Satisfied with their search on Ceres, the squadron of scholars suggests subsequently scanning the stationery stacks of sub-basement 17.

The North Pole printing department is busier than ever this close to Christmas, and while The Historians continue their search of this historically significant facility, an Elf operating a very familiar printer beckons you over.

The Elf must recognize you, because they waste no time explaining that the new sleigh launch safety manual updates won't print correctly. Failure to update the safety manuals would be dire indeed, so you offer your services.

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a very specific order. The notation X|Y means that if both page number X and page number Y are to be produced as part of an update, page number X must be printed at some point before page number Y.

The Elf has for you both the page ordering rules and the pages to produce in each update (your puzzle input), but can't figure out whether each update has the pages in the right order.

For example:

47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
The first section specifies the page ordering rules, one per line. The first rule, 47|53, means that if an update includes both page number 47 and page number 53, then page number 47 must be printed at some point before page number 53. (47 doesn't necessarily need to be immediately before 53; other pages are allowed to be between them.)

The second section specifies the page numbers of each update. Because most safety manuals are different, the pages needed in the updates are different too. The first update, 75,47,61,53,29, means that the update consists of page numbers 75, 47, 61, 53, and 29.

To get the printers going as soon as possible, start by identifying which updates are already in the right order.

In the above example, the first update (75,47,61,53,29) is in the right order:

75 is correctly first because there are rules that put each other page after it: 75|47, 75|61, 75|53, and 75|29.
47 is correctly second because 75 must be before it (75|47) and every other page must be after it according to 47|61, 47|53, and 47|29.
61 is correctly in the middle because 75 and 47 are before it (75|61 and 47|61) and 53 and 29 are after it (61|53 and 61|29).
53 is correctly fourth because it is before page number 29 (53|29).
29 is the only page left and so is correctly last.
Because the first update does not include some page numbers, the ordering rules involving those missing page numbers are ignored.

The second and third updates are also in the correct order according to the rules. Like the first update, they also do not include every page number, and so only some of the ordering rules apply - within each update, the ordering rules that involve missing page numbers are not used.

The fourth update, 75,97,47,61,53, is not in the correct order: it would print 75 before 97, which violates the rule 97|75.

The fifth update, 61,13,29, is also not in the correct order, since it breaks the rule 29|13.

The last update, 97,13,75,29,47, is not in the correct order due to breaking several rules.

For some reason, the Elves also need to know the middle page number of each update being printed. Because you are currently only printing the correctly-ordered updates, you will need to find the middle page number of each correctly-ordered update. In the above example, the correctly-ordered updates are:

75,47,61,53,29
97,61,53,29,13
75,29,13
These have middle page numbers of 61, 53, and 29 respectively. Adding these page numbers together gives 143.

Of course, you'll need to be careful: the actual list of page ordering rules is bigger and more complicated than the above example.

Determine which updates are already in the correct order. What do you get if you add up the middle page number from those correctly-ordered updates?
```

In [123]:
import re
import sys
sys.path.append("..")

from common_processing import read_lines, read_as_character_array,\
    read_columns, read_diagonals

test_path = "test.txt"
data_path = "data.txt"

data = read_lines(test_path)
data

['47|53',
 '97|13',
 '97|61',
 '97|47',
 '75|29',
 '61|13',
 '75|53',
 '29|13',
 '97|29',
 '53|29',
 '61|53',
 '97|53',
 '61|29',
 '47|13',
 '75|47',
 '97|75',
 '47|61',
 '75|61',
 '47|29',
 '75|13',
 '53|13',
 '',
 '75,47,61,53,29',
 '97,61,53,29,13',
 '75,29,13',
 '75,97,47,61,53',
 '61,13,29',
 '97,13,75,29,47']

In [124]:
# Split rules from updates:

raw_rules = []
updates = []
add_rules = True
for line in data:
    if line == "":
        add_rules = False
        continue

    if add_rules:
        raw_rules.append(line)
        continue

    updates.append(line.split(","))

raw_rules, updates

(['47|53',
  '97|13',
  '97|61',
  '97|47',
  '75|29',
  '61|13',
  '75|53',
  '29|13',
  '97|29',
  '53|29',
  '61|53',
  '97|53',
  '61|29',
  '47|13',
  '75|47',
  '97|75',
  '47|61',
  '75|61',
  '47|29',
  '75|13',
  '53|13'],
 [['75', '47', '61', '53', '29'],
  ['97', '61', '53', '29', '13'],
  ['75', '29', '13'],
  ['75', '97', '47', '61', '53'],
  ['61', '13', '29'],
  ['97', '13', '75', '29', '47']])

In [125]:
""" 
approach

Update = [x0, x1, x2, ... xn]

rule = xi|xj where xj must always pressent after xi

take an update, check all rules involving x0
does any xn break the update? (is there a rule such that xn|x0 for xn in update[1:]?)

if so, return FALSE (update is not valid)

if not, run the code again with update[1:]

if len(update) == 1, then return TRUE
"""

def parse_rules(raw_rules: list[str]) -> list[tuple]:
    """ 
    create a dictionary with rules.

     NOTE!!! 

    This only works if we have a single rule per key, not the case in dataset

    """
    rule_pairs = [rule.split("|") for rule in raw_rules]

    rules = {}

    for k, v in rule_pairs:
        if k in rules.keys():
            rules[k].append(v)
        else:
            rules[k] = [v]

    return rules

rules = parse_rules(raw_rules)
rules

{'47': ['53', '13', '61', '29'],
 '97': ['13', '61', '47', '29', '53', '75'],
 '75': ['29', '53', '47', '61', '13'],
 '61': ['13', '53', '29'],
 '29': ['13'],
 '53': ['29', '13']}

In [129]:

def check_rules_for(x0, candidates, rules=rules) -> bool:
    """ 
    check rules and return wheter x0 is always before all candidates

    return True (valid) as default, change it to False if
    for any candidate xn there is a rull such that xn|x0
    """

    for cand in candidates:
        if cand in rules.keys() and x0 in rules[cand]:
            print("Exlcuded by rule: ",f"{cand}|{x0}")

            return False

    return True

def check_update(update: list) -> bool:

    if len(update) == 1:
        print("Update exhausted")
        return True

    x0 = update[0]
    candidates = update[1:]

    result = check_rules_for(x0, candidates)

    if result == True:
        return check_update(candidates)
    else:
        return False
    
valid_updates = []
for update in updates:
    if check_update(update):
        valid_updates.append(update)

print("Valid updates: ")
for vu in valid_updates:
    print(vu)


Update exhausted
Exlcuded by rule:  46|69
Update exhausted
Exlcuded by rule:  65|19
Update exhausted
Update exhausted
Update exhausted
Exlcuded by rule:  24|33
Update exhausted
Update exhausted
Update exhausted
Exlcuded by rule:  62|41
Exlcuded by rule:  38|23
Exlcuded by rule:  48|15
Update exhausted
Exlcuded by rule:  73|71
Exlcuded by rule:  36|73
Exlcuded by rule:  15|95
Exlcuded by rule:  83|11
Exlcuded by rule:  72|75
Update exhausted
Exlcuded by rule:  19|75
Exlcuded by rule:  85|58
Exlcuded by rule:  79|98
Exlcuded by rule:  56|75
Update exhausted
Update exhausted
Update exhausted
Update exhausted
Exlcuded by rule:  38|67
Update exhausted
Update exhausted
Exlcuded by rule:  79|41
Exlcuded by rule:  56|54
Exlcuded by rule:  63|12
Exlcuded by rule:  12|57
Update exhausted
Exlcuded by rule:  33|79
Exlcuded by rule:  41|65
Update exhausted
Update exhausted
Exlcuded by rule:  65|44
Update exhausted
Update exhausted
Exlcuded by rule:  82|15
Update exhausted
Update exhausted
Update ex

In [130]:
# Select the middle element and count them
result = 0

for vu in valid_updates:
    result += int(vu[(len(vu) - 1)//2 ])

print(result)

4662


In [131]:
# Try with the whole dataset
data = read_lines(data_path)

raw_rules = []
updates = []
add_rules = True
for line in data:
    if line == "":
        add_rules = False
        continue

    if add_rules:
        raw_rules.append(line)
        continue

    updates.append(line.split(","))

print(len(updates)," Updates")
print(len(raw_rules)," rules")

rules = parse_rules(raw_rules)

valid_updates = []
for update in updates:
    if check_update(update):
        valid_updates.append(update)
print(len(valid_updates)," Valid updates")

# Select the middle element and count them
result = 0

for vu in valid_updates:
    result += int(vu[(len(vu) - 1)//2 ])

print("Sum of the middle elements",result)

192  Updates
1176  rules
Update exhausted
Exlcuded by rule:  46|69
Update exhausted
Exlcuded by rule:  65|19
Update exhausted
Update exhausted
Update exhausted
Exlcuded by rule:  24|33
Update exhausted
Update exhausted
Update exhausted
Exlcuded by rule:  62|41
Exlcuded by rule:  38|23
Exlcuded by rule:  48|15
Update exhausted
Exlcuded by rule:  73|71
Exlcuded by rule:  36|73
Exlcuded by rule:  15|95
Exlcuded by rule:  83|11
Exlcuded by rule:  72|75
Update exhausted
Exlcuded by rule:  19|75
Exlcuded by rule:  85|58
Exlcuded by rule:  79|98
Exlcuded by rule:  56|75
Update exhausted
Update exhausted
Update exhausted
Update exhausted
Exlcuded by rule:  38|67
Update exhausted
Update exhausted
Exlcuded by rule:  79|41
Exlcuded by rule:  56|54
Exlcuded by rule:  63|12
Exlcuded by rule:  12|57
Update exhausted
Exlcuded by rule:  33|79
Exlcuded by rule:  41|65
Update exhausted
Update exhausted
Exlcuded by rule:  65|44
Update exhausted
Update exhausted
Exlcuded by rule:  82|15
Update exhausted
U

10604 is too high